In order to run this, you should have vector database `(vector_db_)` already stored as .zip. You can download it using the link
https://drive.google.com/drive/folders/1o2Y-BLNWNFT5zesysd2n3QZPtN0Jro3I?usp=drive_link

In [1]:
# After running this code the KERNEL needs to be restarted

!pip install chromadb==0.5.5 langchain-chroma==0.1.2 langchain==0.2.11 langchain-community==0.2.10 langchain-text-splitters==0.2.2 langchain-groq==0.1.6 transformers==4.43.2 sentence-transformers==3.0.1 unstructured==0.15.0 unstructured[pdf]==0.15.0


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 2.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 12.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.0/117.0 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.2/42.2 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.0/42.0 kB 2.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.5/48.5 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 584.3/584.3 kB 29.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 990.3/990.3 kB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━

Run the code snippet below it has every library included.

In [3]:
import os
import pandas as pd
from transformers import pipeline

# Langchain modules
from langchain.document_loaders import UnstructuredFileLoader, PyPDFDirectoryLoader, TextLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import TextLoader
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.llms import HuggingFacePipeline
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
    SentenceTransformersTokenTextSplitter,
    TokenTextSplitter,
)
from langchain.chains.question_answering import load_qa_chain
from langchain.vectorstores import Chroma
from langchain_groq import ChatGroq

# Set API Key for Groq
os.environ["GROQ_API_KEY"] = "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"


# Download Sentence Transformers Embedding From Hugging Face
embeddings = HuggingFaceEmbeddings(model_name ='sentence-transformers/all-MiniLM-L6-v2')
# embeddings = HuggingFaceEmbeddings() # Make sure you use the same embedders as you used to embed the original dataset into vector database.


<ipython-input-3-da566b941860>:28: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 0.3.0. An updated version of the class exists in the langchain-huggingface package and should be used instead. To use it run `pip install -U langchain-huggingface` and import as `from langchain_huggingface import HuggingFaceEmbeddings`.
  embeddings = HuggingFaceEmbeddings(model_name ='sentence-transformers/all-MiniLM-L6-v2')
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

There are two ways to read and predict in the model, Manual and Automatic

# Manual entry of vector database

In [ ]:
import os
from zipfile import ZipFile

# Directory containing the zip files
zip_dir = "/content/"
extract_path = "/content/"  # Extraction path

# Iterate through all files in the directory
for file_name in os.listdir(zip_dir):
    if file_name.endswith(".zip"):  # Check if the file is a .zip file
        zip_path = os.path.join(zip_dir, file_name)
        print(f"Extracting {zip_path}...")

        # Unzip the file
        with ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
            print(f"Extraction completed for {file_name}!")


Extracting /content/vectordb_book2.zip...
Extraction completed for vectordb_book2.zip!
Extracting /content/vector_db_auto_parts.zip...
Extraction completed for vector_db_auto_parts.zip!
Extracting /content/vectordb_book1.zip...
Extraction completed for vectordb_book1.zip!


In [ ]:
# Retrieving the first vector database
persist_directory_1 = "/content/content/auto_book_1"
vectordb_1 = Chroma(
    persist_directory=persist_directory_1,
    embedding_function=embeddings
)

# Retrieving the second vector database
persist_directory_2 = "/content/content/dheer_auto_parts_db"
vectordb_2 = Chroma(
    persist_directory=persist_directory_2,
    embedding_function=embeddings
)

# Retrieving the third vector database
persist_directory_3 = "/content/content/auto_book_2"
vectordb_3 = Chroma(
    persist_directory=persist_directory_3,
    embedding_function=embeddings
)
# Create retrievers for both vector databases
retriever_1 = vectordb_1.as_retriever()
retriever_2 = vectordb_2.as_retriever()
retriever_3 = vectordb_3.as_retriever()

# Define a combined retriever function
def combined_retriever(query):
    # Retrieve documents from both retrievers
    results_1 = retriever_1.get_relevant_documents(query)
    results_2 = retriever_2.get_relevant_documents(query)
    results_3 = retriever_3.get_relevant_documents(query)

    # Combine results (you can sort or filter if needed)
    combined_results = results_1 + results_2 + results_3
    return combined_results

# Initialize the LLM
llm = ChatGroq(
    model="llama-3.2-90b-vision-preview",
    temperature=0
)

# Load a question-answering chain using the "stuff" chain type
qa_chain = load_qa_chain(llm, chain_type="map_reduce")


In [ ]:
# Create a prompt template
chatbot_prompt = """
You are an automotive parts assistant. When a user asks about their vehicle, you will refer to relevant documents and provide guidance in a concise, clear manner. Your goal is to stay under 1,000 tokens for each response, including all necessary details.

Here are the steps to follow:

1. **Identify the Vehicle and Parts**: Determine what part the user is asking about based on their question. If it’s unclear, ask clarifying questions to understand the model, make, and year of the vehicle.

2. **Provide Pricing**: Always provide the price for the part requested. If you don’t have the exact year of the vehicle the user asks about, provide the price for the earliest year available in your database and inform the user that you cannot help with the exact year requested. You should always mention the earliest year available, even if the part is not available for the requested year.

3. **Include Related Subcategories**: If the part requested falls under a category that has subcategories (e.g., "engine parts" has "fuel injectors", "oil filters", etc.), list those subcategories with their prices, if available.

4. **Mention Fluids**: If the part requested is related to fluids (e.g., oil, transmission fluid), also mention the fluids associated with the part and their availability/price.

5. **Maintain Chat Context**: Keep track of previous conversations and refer to them when necessary to provide consistent follow-up answers. For example, if the user has already asked about a part and later asks about another part from the same vehicle, refer back to previous details such as model, year, or part-related info.

6. **Structure the Response**:
   - Begin by acknowledging the vehicle type and confirming details, especially if the model year or make was mentioned.
   - Provide the price for the part.
   - List any relevant subcategories and related fluids.
   - If no exact match for the year is found, state the earliest year available in the database and explain the limitation.

**Keep responses concise, with no more than 1,000 tokens. If the response exceeds the token limit, trim unnecessary details.**

Conversation so far:
{chat_history}

User's question:
{user_input}

"""

# Define the chatbot function
def chatbot():
    print("Welcome to the Car Issue Chatbot! Type 'exit' to end the conversation.")

    # Initialize chat history
    chat_history = []

    while True:
        # Get user input
        query = input("\n **You**: ")

        # Exit condition
        if query.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        # Automatically add the user message to the chat history
        chat_history.append({"role": "user", "content": query})

        # Create prompt for the current conversation context
        prompt = chatbot_prompt.format(
            chat_history="\n".join([f"{message['role'].capitalize()}: {message['content']}" for message in chat_history]),
            user_input=query
        )

        # Retrieve documents from both vector databases
        combined_results = combined_retriever(query)

        # Pass the combined results to the chain
        response = qa_chain.run(
            input_documents=combined_results,
            question=prompt  # Passing the user query directly
        )

        # Print the result
        print(f"\n **SBG**: {response}")

        # Add assistant's response to chat history automatically
        chat_history.append({"role": "assistant", "content": response})

# Run the chatbot
chatbot()

Welcome to the Car Issue Chatbot! Type 'exit' to end the conversation.

 **You**: I have 2011 toyota rav 4 and my brakes are not working

 **SBG**: I understand you're having issues with your 2011 Toyota RAV4's brakes. I've found some relevant information regarding brake performance. According to the document, "Some brakes stop well when they are cold." This might be related to your issue.

For a 2011 Toyota RAV4, the brake pads cost around $50-$70 per set. However, I would recommend checking the brake rotors as well, which cost around $200-$300 per set.

Some related subcategories for brake parts include:

- Brake pads ($50-$70 per set)
- Brake rotors ($200-$300 per set)
- Brake fluid (available, $10-$20 per liter)

Please note that these prices are estimates and may vary depending on the specific part and brand.

Additionally, I would like to ask if you have checked the brake fluid level and condition in your vehicle?

 **You**: waht about hydrolics ?

 **SBG**: I understand you're i

# Automatic entry of vector database


## Unziping

In [1]:
import os
from zipfile import ZipFile

# Directory containing the zip files
zip_dir = "/content/"
extract_path = "/content/"  # Extraction path

# Iterate through all files in the directory
for file_name in os.listdir(zip_dir):
    if file_name.endswith(".zip"):  # Check if the file is a .zip file
        zip_path = os.path.join(zip_dir, file_name)
        print(f"Extracting {zip_path}...")

        # Unzip the file
        with ZipFile(zip_path, 'r') as zip_ref:
            zip_ref.extractall(extract_path)
            print(f"Extraction completed for {file_name}!")

Extracting /content/vectordb_book2.zip...
Extraction completed for vectordb_book2.zip!
Extracting /content/vector_db_auto_parts.zip...
Extraction completed for vector_db_auto_parts.zip!
Extracting /content/vectordb_book1.zip...
Extraction completed for vectordb_book1.zip!


## Reading the vector database

In [4]:
import os
from langchain.vectorstores import Chroma

# Directory containing vector databases
vector_db_dir = "/content/content/"
embedding_function = embeddings  # Ensure this is defined earlier in your code

# Initialize a list to store retrievers
retrievers = []

# Iterate through all subdirectories in the specified directory
for sub_dir in os.listdir(vector_db_dir):
    full_path = os.path.join(vector_db_dir, sub_dir)
    if os.path.isdir(full_path):  # Check if it's a directory
        print(f"Loading vector database from: {full_path}")

        # Load the vector database
        vectordb = Chroma(
            persist_directory=full_path,
            embedding_function=embedding_function
        )

        # Create a retriever and add it to the list
        retrievers.append(vectordb.as_retriever())

# Define a combined retriever function
def combined_retriever(query):
    combined_results = []
    for retriever in retrievers:
        # Retrieve documents from each retriever
        combined_results.extend(retriever.get_relevant_documents(query))

    return combined_results

# Example Usage
query = "What is the price of a fuel injector for 2005 toyota rav 4 ?"
results = combined_retriever(query)
print(f"Retrieved {len(results)} documents.")

# Initialize the LLM
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0
)

# Load a question-answering chain using the "stuff" chain type
qa_chain = load_qa_chain(llm, chain_type="map_reduce")


Loading vector database from: /content/content/auto_book_2


<ipython-input-4-16a52746420a>:18: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 0.4. An updated version of the class exists in the langchain-chroma package and should be used instead. To use it run `pip install -U langchain-chroma` and import as `from langchain_chroma import Chroma`.
  vectordb = Chroma(


Loading vector database from: /content/content/dheer_auto_parts_db
Loading vector database from: /content/content/auto_book_1


<ipython-input-4-16a52746420a>:31: LangChainDeprecationWarning: The method `BaseRetriever.get_relevant_documents` was deprecated in langchain-core 0.1.46 and will be removed in 1.0. Use invoke instead.
  combined_results.extend(retriever.get_relevant_documents(query))


Retrieved 12 documents.


## Chatbot finetuning and chatting

In [5]:
# Create a prompt template
chatbot_prompt = """
You are an intelligent and efficient **Automotive Parts and Fluids Assistant**. Your goal is to assist customers by identifying the most relevant parts and fluids for their vehicles based on their questions or descriptions. Always prioritize clarity, accuracy, and customer satisfaction. Follow these guidelines:
If a customer says something generic like, “My car isn’t starting,” the issue could be from multiple causes such as the alternator, battery, starter, oil leaks, etc. the chatbot would ask clarifying questions which are very helpful for initial inspection such as “Does the engine turn over?” or “Did you notice oil drips on your driveway?” or “Did the car stop after reaching 90 km/h?”. The chatbot would then provide an estimate for the mandatory "No Start" inspection and, based on the customer’s responses, suggest the top three most likely repair orders along with the total cost. The model would continuously optimize itself by learning from feedback given by auto service advisors on the repair orders.

On the auto parts side, the bot could easily provide customers with the parts they ask for specifically when customers are unsure of the exact name of a part, they can describe the parts and based on it the chatbot would give top 3 parts that matches with description given by customer. It would also show availability, different manufacturers and prices, stock levels, and warranty details in real-time.
**Keep responses concise, with no more than 1,000 tokens. If the response exceeds the token limit, trim unnecessary details.**

Conversation so far:
{chat_history}

User's question:
{user_input}
"""

# Define the chatbot function
def chatbot():
    print("Welcome to the Car Issue Chatbot! Type 'exit' to end the conversation.")

    # Initialize chat history
    chat_history = []

    while True:
        # Get user input
        query = input("\n **You**: ")

        # Exit condition
        if query.lower() == "exit":
            print("Chatbot: Goodbye!")
            break

        # Automatically add the user message to the chat history
        chat_history.append({"role": "user", "content": query})

        # Create prompt for the current conversation context
        prompt = chatbot_prompt.format(
            chat_history="\n".join([f"{message['role'].capitalize()}: {message['content']}" for message in chat_history]),
            user_input=query
        )

        # Retrieve documents from both vector databases
        combined_results = combined_retriever(query)

        # Pass the combined results to the chain
        response = qa_chain.run(
            input_documents=combined_results,
            question=prompt  # Passing the user query directly
        )

        # Print the result
        print(f"\n **SBG**: {response}")

        # Add assistant's response to chat history automatically
        chat_history.append({"role": "assistant", "content": response})

# Run the chatbot
chatbot()

Welcome to the Car Issue Chatbot! Type 'exit' to end the conversation.

 **You**: My car is not starting\


<ipython-input-5-76b572d334d4>:45: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 0.3.0. Use invoke instead.
  response = qa_chain.run(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/665 [00:00<?, ?B/s]


 **SBG**: To help diagnose the issue, can you tell me if the engine turns over when you try to start it?

 **You**: No the engine won't start 


Token indices sequence length is longer than the specified maximum sequence length for this model (1863 > 1024). Running this sequence through the model will result in indexing errors



 **SBG**: Based on your description, it seems like the engine isn't turning over, which could indicate a problem with the battery, starter motor, or ignition system. 

To further diagnose the issue, I'd like to ask a few more questions:

1. Have you checked the battery terminals for corrosion or loose connections?
2. Are the headlights or interior lights turning on when you try to start the engine?
3. Have you recently replaced the battery or starter?

Please let me know your responses, and I'll provide further guidance.

 **You**: exit
Chatbot: Goodbye!
